In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

In [3]:
train = pd.read_csv('/content/drive/MyDrive/PyTorch/final_train.csv')

In [4]:
train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,is_event,family,class,perishable,dcoilwtico
0,2017-01-01,25,99197,1.0,0,1,12,43,0,53.19
1,2017-01-01,25,103665,7.0,0,1,5,184,1,53.19
2,2017-01-01,25,105574,1.0,0,1,12,30,0,53.19
3,2017-01-01,25,105857,4.0,0,1,12,62,0,53.19
4,2017-01-01,25,106716,2.0,0,1,12,19,0,53.19


In [5]:
cut_train = train.loc[train['date'] > '2017-05-31']
test_len = len(train.loc[train['date'] > '2017-07-31'])
border = len(cut_train) - test_len

In [6]:
border

6448195

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Эта обработка основана на этой статье
# https://www.crosstab.io/articles/time-series-pytorch-lstm
# видела похожие решения и в других источниках
# этот код у меня не сработал, и я его немного переделала
# (Grocery_2)


class Grocery():
    
    def __init__(self, dataset, target, scalar, sequence=5):
        self.dataset= dataset
        self.target = target
        self.sequence = sequence
        self.scalar = scalar()
        self.y = torch.tensor(self.dataset[self.target].to_numpy(dtype='float64')).unsqueeze(1)     
        self.X = self.scalar.fit_transform(torch.tensor(self.dataset.drop(self.target, axis=1).to_numpy(dtype='float64')))
       
        
    def __len__(self):
        return len(self.dataset)
    
    
    def __getitem__(self, i): 
        if i >= self.sequence - 1:
            i_start = i - self.sequence + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat((self.sequence - i - 1), 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]
        

In [7]:
class Grocery_2():
    
    def __init__(self, dataset, target, scalar, sequence=5):
        self.dataset= dataset
        self.target = target
        self.sequence = sequence
        self.scalar = scalar()
        self.y = torch.tensor(self.dataset[self.target].to_numpy(dtype='float64')).unsqueeze(1)     
        self.X = self.scalar.fit_transform(torch.tensor(self.dataset.drop(self.target, axis=1).to_numpy(dtype='float64')))
       
        
    def __len__(self):
        return len(self.dataset)
    
    
    def __getitem__(self, i): 
        if i >= self.sequence - 1:
            i_start = i - self.sequence + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.sequence - i  # просто отстригаю нужную длинну от начала данных, а не повторяю первую строку данных
            # как это реализовано в коде выше. Лучше ничего не придумала. Вопрос тут только один. Смысл в такой обработке с шагом есть?
            # Это улучшает предсказательную способность модели или просто ресурсы переводим зря?
            x = self.X[0:(i + padding), :]
        return x, self.y[i]

In [13]:
cut_train['date'] = pd.to_datetime(cut_train['date'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
f_train = Grocery_2(cut_train[:border],'unit_sales', StandardScaler, 2)
f_test = Grocery_2(cut_train[border:],'unit_sales', StandardScaler, 2)

In [15]:
len(f_train), len(f_test)

(6448195, 1570968)

In [16]:
train_loader = torch.utils.data.DataLoader(f_train,
                          batch_size=1000,
                          shuffle=True,
                          num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(f_test,
                          batch_size=1000,
                          shuffle=False,
                          num_workers=1, 
                          drop_last=True)

In [17]:
class RNN(nn.Module):
    
    def __init__(self, num_features, hidden_dim, num_layers=1):
        super().__init__()
        self.num_features = num_features
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size=self.num_features,
                      hidden_size=self.hidden_dim,
                      batch_first=True,
                      num_layers=self.num_layers)
        self.lin = nn.Linear(in_features=self.hidden_dim, 
                            out_features=1)
        
    def forward(self, x):
        #h0 = torch.zeros(self.num_layers,
                         #x.shape[0],
                         #self.hidden_dim).requires_grad_()
        #c0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_dim).requires_grad_()
        out, ht= self.rnn(x)
        out = self.lin(out[:, -1])
        return out
    

In [18]:
model = RNN(9, 1000)

In [19]:
print(model)

RNN(
  (rnn): RNN(9, 1000, batch_first=True)
  (lin): Linear(in_features=1000, out_features=1, bias=True)
)


In [20]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [21]:
def train_model(data_loader, model, loss_function, optimizer, epochs_num, batch_len):
    for epoch in tqdm(range(epochs_num)):
        
        num_batches = len(data_loader)
        total_loss = 0
        model.train()
        model.double()
        for ind, (X, y) in enumerate(data_loader):
            output = model(X.double())
            loss = loss_function(output, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            if ind % batch_len == 0:
              current_loss, current_len = loss.item() ,ind * len(X)
              print(f'Batch_loss: {current_loss}, num_butches {ind} / {num_batches} ')

        avg_loss = total_loss / num_batches     
        print(f"Epoch: {epoch + 1} Train loss: {avg_loss}")
        
        
        # model.eval()
        # loss_accumed = 0
        # for X, y in train_loader:
        #     output = model(X.double())
        #     loss = loss_function(output, y)
        #     loss_accumed += loss.item()
        # eval_loss = loss_accumed / num_batches
        # print(f"Epoch: {epoch + 1} Test_loss: {eval_loss}")

In [23]:
# Все-таки  прогнала и на этом варианте предобработки
# прироста по качеству не увидела, но ошибка так сильно не скачет
train_model(train_loader, model, loss, optimizer, 1, 1000)

  0%|          | 0/1 [00:00<?, ?it/s]

Batch_loss: 816.0146644713894, num_butches 0 / 6448 
Batch_loss: 201.77277936479473, num_butches 1000 / 6448 
Batch_loss: 235.20350806598861, num_butches 2000 / 6448 
Batch_loss: 401.63356553606667, num_butches 3000 / 6448 
Batch_loss: 125.38683599307618, num_butches 4000 / 6448 
Batch_loss: 422.48420925402326, num_butches 5000 / 6448 
Batch_loss: 192.73508150869623, num_butches 6000 / 6448 


100%|██████████| 1/1 [39:43<00:00, 2383.89s/it]

Epoch: 1 Train loss: 611.4468275689659


In [27]:
def test_model(data_loader, model, loss_function):

    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")

In [28]:
test_model(test_loader, model, loss)

Test loss: 471.655940996818
